In [6]:
import polars as pl
import os

In [4]:
df_por_fecha = pl.read_parquet('../DATASETS/datos_por_alcaldia_y_fecha.parquet.gzip')
df_por_fecha

fecha_reporte,alcaldia,total_fugas
datetime[ms],str,u32
2021-01-01 00:00:00,"""TLALPAN""",2
2021-01-01 00:00:00,"""CUAUHTEMOC""",2
2021-01-01 00:00:00,"""XOCHIMILCO""",1
2021-01-01 00:00:00,"""GUSTAVO A. MADERO""",1
2021-01-01 00:00:00,"""ALVARO OBREGON""",3
…,…,…
2024-01-31 00:00:00,"""IZTAPALAPA""",11
2024-02-01 00:00:00,"""MIGUEL HIDALGO""",1
2024-02-01 00:00:00,"""GUSTAVO A. MADERO""",2


In [7]:
user = os.getenv("POSTGRES_USER", "postgres")
password = os.getenv("POSTGRES_PASSWORD", "postgres")
host = os.getenv("POSTGRES_HOST", "localhost")
port = os.getenv("POSTGRES_PORT", "5432")
db = os.getenv("POSTGRES_DB", "sacmex")

In [11]:
uri = f'postgresql://{user}:{password}@{host}:{port}/{db}'

df_por_fecha.write_database(table_name="fugas_por_delegacion", connection=uri, if_table_exists='append')

500

In [12]:
df = pl.read_parquet('../DATASETS/combinado_fugas.parquet.gzip')
# df = df.reset_index(names='fecha_reporte')
df.columns = ['total_fugas', 'target', 'fecha_reporte']
df

total_fugas,target,fecha_reporte
f64,str,datetime[ns]
17.0,"""ORIGEN""",2021-01-01 00:00:00
79.0,"""ORIGEN""",2021-01-02 00:00:00
84.0,"""ORIGEN""",2021-01-03 00:00:00
222.0,"""ORIGEN""",2021-01-04 00:00:00
83.0,"""ORIGEN""",2021-01-05 00:00:00
…,…,…
3496.0,"""PRONOSTICO""",2024-02-25 00:00:00
3910.0,"""PRONOSTICO""",2024-02-26 00:00:00
2957.0,"""PRONOSTICO""",2024-02-27 00:00:00


In [14]:
df.write_database(table_name="fugas_por_fecha", connection=uri, if_table_exists='append')

155

In [17]:
simulated_data = pl.read_parquet('../DATASETS/datos_simulados.parquet.gzip')
simulated_data

folio,fecha_registro_incidente,id_reporte,fecha_reporte,hora_reporte,clasificacion,tipo_de_falla,medio_recepcion,alcaldia_catalogo,colonia_catalogo,latitud,longitud,day,month,year,status
str,datetime[ns],str,date,time,str,str,str,str,str,f64,f64,i64,i64,i64,str
"""I-20230857-3334""",2024-08-14 04:16:45,"""R-20231207-1480""",2024-08-10,04:16:45,"""Agua Potable""","""Fuga""","""Redes Sociales (Instagram)""","""ALVARO OBREGON""","""El Piru 2da Ampliacion""",19.445022,-99.083052,10,8,2024,"""SOLUCIONANDO"""
"""I-20230376-5224""",2024-08-03 14:24:40,"""R-20230136-8419""",2024-07-31,14:24:40,"""Agua Tratada""","""Fuga""","""Locatel folio""","""TLAHUAC""","""La Planta""",19.236461,-99.292003,31,7,2024,"""ATENDIENDO"""
"""I-20230178-4392""",2024-09-28 17:05:09,"""R-20230324-2281""",2024-09-19,17:05:09,"""Agua Tratada""","""Fuga""","""Redes Sociales (Facebook)""","""CUAJIMALPA DE MORELOS""","""PROVIDENCIA""",19.313443,-99.046494,19,9,2024,"""EN PROCESO"""
"""I-20230857-5264""",2024-07-21 12:14:30,"""R-20230523-7151""",2024-07-19,12:14:30,"""Agua Potable""","""Fuga""","""Jefatura de Gobierno/Base Cúsp…","""CUAUHTEMOC""","""HIPODROMO CONDESA""",19.433645,-99.154349,19,7,2024,"""RESUELTO"""
"""I-20230165-1876""",2024-03-19 21:18:53,"""R-20230729-3049""",2024-03-16,21:18:53,"""Agua Potable""","""Fuga""","""Redes Sociales (Instagram)""","""Álvaro Obregón""","""SANTA MARIA AZTAHUACAN (EJ) II""",19.27877,-99.071977,16,3,2024,"""EN REVISIÓN"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""I-20230410-6518""",2024-11-23 21:57:18,"""R-20231208-7639""",2024-11-19,21:57:18,"""Agua Tratada""","""Fuga""","""Ciudadano (Call Center)""","""BENITO JUAREZ""","""Barrio Santa Lucia""",19.204625,-99.15188,19,11,2024,"""ATENDIENDO"""
"""I-20230854-2451""",2024-07-14 16:20:20,"""R-20230234-5601""",2024-07-10,16:20:20,"""Agua Tratada""","""Fuga""","""Personal Directivo de SACMEX""","""MILPA ALTA""","""LA ESMERALDA III""",19.336959,-99.162828,10,7,2024,"""RESUELTO"""
"""I-20230808-7030""",2024-11-04 20:44:44,"""R-20230464-8582""",2024-11-03,20:44:44,"""Agua Potable""","""Fuga""","""Jefatura de Gobierno/Base Cúsp…","""GUSTAVO A. MADERO""","""PEDREGAL DE STO DOMINGO IV""",19.298391,-99.181271,3,11,2024,"""RESUELTO"""


In [18]:
simulated_data.write_database(table_name="reportes_fugas", connection=uri, if_table_exists='append')

1000